# O Problema
Sliding Puzzle - Bloco Deslizante

In [1]:
# !wget -qq https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif
from IPython.display import Image
Image(url='https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif',width=200)

# Resolver o quebra-cabeças usando Buscas

In [2]:
import itertools

import numpy as np


possible_moves = [
    (0, -1),  (1, 0), 
    (-1, 0),  (0, 1)
]


class State(object):
    def __init__(self, matrix):
        # Knowledge
        self.matrix = matrix

        # Representation
        self.representation = self.encode(matrix)

        # A* total cost
        self.F = int()

        # Moves to get to State
        self.moves = None
    
    def encode(self, matrix):
        # Flatten the matrix
        array = list(itertools.chain.from_iterable(matrix))

        # Convert to string
        return "".join(str(elem) for elem in array)
    
    def get_empty_position(self):
        # Find empty position (represented by 0)
        line, column = np.where(self.matrix == 0)
        line = line[0]
        column = column[0]

        empty_position = (line, column)

        return empty_position

    def get_moves(self):
        empty_position = self.get_empty_position()
        line, column = empty_position

        # Get positions that can move
        moves = list()

        for horizontal, vertical in possible_moves:
            position = ((line + vertical), (column + horizontal))

            # Verify if all index are smaller than 3
            if all(-1 < pos < 3 for pos in position):
                moves.append((position))
        
        return moves, empty_position
    
    def __del__(self):
        # Free the memory allocated by the matrix
        del self.matrix

        # Free the memory allocated by the moves array
        del self.moves
    
    def __lt__(self, other):
        # Use F to compare states
        return self.F > other.F



def swap_matrix(matrix, position1, position2):
    new_matrix = matrix.copy()

    new_matrix[position1], new_matrix[position2] = new_matrix[position2], new_matrix[position1]

    return new_matrix


def print_moves(solving_moves, matrix_input):
    matrix = matrix_input.copy()
    
    for index in range(len(solving_moves) - 1):
        empty_pos = solving_moves[index]
        new_pos = solving_moves[index + 1]

        print(matrix, '\n')
        matrix[empty_pos], matrix[new_pos] = matrix[new_pos], matrix[empty_pos]
    
    print(matrix)


## Busca A*

In [3]:
# This functions insert the state to the open_states array according to F.
from bisect import insort


solutions = ("123456780", "012345678")


def solve_sliding_puzzle_a_star(state, heuristic):
    visited_states = set()
    open_states = list()

    # Verify if initial state is the solution
    if state.representation in solutions:
        # The solution was found
        return state.moves
    
    # Insert initial state to open_states
    insort(open_states, state)

    while len(open_states) != 0:
        # print([(o.F, o.representation) for o in open_states])

        current_state = open_states.pop()

        # print(f"Current:\n{current_state.matrix}\n")

        if current_state.representation in visited_states:
            # Free up state memory
            del current_state
            continue

        visited_states.add(current_state.representation)

        current_moves, empty_pos = current_state.get_moves()

        for move_pos in current_moves:
            # Swap empty position to move position
            new_matrix = swap_matrix(current_state.matrix, move_pos, empty_pos)

            new_state = State(new_matrix)

            # Verify if state has already been visited
            if new_state.representation in visited_states:
                # Free up state memory
                del new_state
                continue
            
            # Add move position to moves
            new_moves = current_state.moves.copy()

            new_moves.append(move_pos)

            if new_state.representation in solutions:
                # The solution was found
                return new_moves

            new_state.moves = new_moves

            # Calculate F for A*
            G = len(new_moves)
            H = heuristic(new_state, solutions)

            new_state.F = G + H

            # print(new_state.matrix, new_state.F, move_pos, empty_pos)

            # Insert based on F
            insort(open_states, new_state)

        # Free up memory
        del current_state

    return None

### Usando a distância de Levenshtein como heurística para o A*


In [4]:
!pip install python-Levenshtein

     |████████████████████████████████| 50 kB 153 kB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp38-cp38-linux_x86_64.whl size=166084 sha256=854234909c6df25e7aa5cbd9281b27d82c5b7e07230a8064838c4bb5726b3c08
  Stored in directory: /home/arthur/.cache/pip/wheels/d7/0c/76/042b46eb0df65c3ccd0338f791210c55ab79d209bcc269e2c7
Successfully built python-Levenshtein


In [5]:
import Levenshtein

def levenshtein_heuristic(state, solutions):
    # Get min distance between state representation and any of the solutions
    return min(
      Levenshtein.distance(state.representation, solution) for solution in solutions
    )

matrix = np.array(
    [[2, 3, 6],
     [1, 4, 8],  
     [7, 5, 0]]
)

initial_state = State(matrix.copy())
initial_position = [initial_state.get_empty_position()]

initial_state.moves = initial_position

solving_moves = solve_sliding_puzzle_a_star(initial_state, levenshtein_heuristic)

print(f"Number of moves to solve the problem: {len(solving_moves) - 1}")


Number of moves to solve the problem: 8


In [6]:
print_moves(solving_moves, matrix)

[[2 3 6]
 [1 4 8]
 [7 5 0]] 

[[2 3 6]
 [1 4 0]
 [7 5 8]] 

[[2 3 0]
 [1 4 6]
 [7 5 8]] 

[[2 0 3]
 [1 4 6]
 [7 5 8]] 

[[0 2 3]
 [1 4 6]
 [7 5 8]] 

[[1 2 3]
 [0 4 6]
 [7 5 8]] 

[[1 2 3]
 [4 0 6]
 [7 5 8]] 

[[1 2 3]
 [4 5 6]
 [7 0 8]] 

[[1 2 3]
 [4 5 6]
 [7 8 0]]


### Usando os elementos em sequencia como heurística para A*

In [7]:
def sequences_heuristic(state_moves, solutions):
    elements = list(itertools.chain.from_iterable(matrix))
    sequence_counter = 0

    for index in range(1, len(elements)):
        previous_element = elements[index -1]
        current_element = elements[index]

        if current_element == previous_element + 1:
            sequence_counter += 1
    
    return sequence_counter


matrix = np.array(
    [[2, 3, 6],
     [1, 4, 8],  
     [7, 5, 0]]
)

initial_state = State(matrix.copy())
initial_position = [initial_state.get_empty_position()]

initial_state.moves = initial_position

solving_moves = solve_sliding_puzzle_a_star(initial_state, sequences_heuristic)

print(f"Number of moves to solve the problem: {len(solving_moves) - 1}")


Number of moves to solve the problem: 8


In [8]:
print_moves(solving_moves, matrix)

[[2 3 6]
 [1 4 8]
 [7 5 0]] 

[[2 3 6]
 [1 4 0]
 [7 5 8]] 

[[2 3 0]
 [1 4 6]
 [7 5 8]] 

[[2 0 3]
 [1 4 6]
 [7 5 8]] 

[[0 2 3]
 [1 4 6]
 [7 5 8]] 

[[1 2 3]
 [0 4 6]
 [7 5 8]] 

[[1 2 3]
 [4 0 6]
 [7 5 8]] 

[[1 2 3]
 [4 5 6]
 [7 0 8]] 

[[1 2 3]
 [4 5 6]
 [7 8 0]]


## Discusta sobre o desempenho dos métodos em questões de:


1.   Consumo de memória

Apesar da necessidade de armazenar uma lista de estados abertos, as estratégias de busca com informação apresentam um consumo de memória muito mais baixo do que nas buscas sem informação. As heurísticas adotadas apresentaram, em seu pior caso, o mesmo consumo de memória que as buscas sem informação.

2.   Processamento

As buscas com informação também apresentam requisitos de processamento mais baixos que as buscas sem informação. Nesse tipo de busca, existe a possibilidade da heurística apresentar uma complexidade que supere possíveis ganhos da estratégia. No exemplo do problema dos blocos deslizantes, as duas heurísticas adotadas se mostram eficientes na sua resolução.

